In [15]:
from everything import *

In [16]:
agg_num=100
aggdf = pd.read_csv(f"aggregated_{agg_num}_std.csv", index_col =False)
dat= aggdf[["acoustic_data", "time_to_failure"]].to_numpy()
# oaggdf = pd.read_csv(f"aggregated_{agg_num}_mean.csv", index_col =False)
# odata = oaggdf[["acoustic_data"]].to_numpy()


In [17]:
#data = np.hstack([odata,dat])

In [18]:
data = dat

In [19]:
from sklearn.model_selection import train_test_split
#maybe normalize about zero?
np.random.seed(42)
ratio = 0.2
num_channels = 1
n = len(data)
sample_size = int(150_000/agg_num)
split_data = np.array(np.split(data[:(n//sample_size)*sample_size], n//sample_size))
# labels are first time to appear 
split_inputs = split_data[:,:,0].reshape(-1, sample_size,1)
split_labels = split_data[:,0,1].reshape(-1,1)
#split_inputs_fft = np.fft.hfft(split_inputs, axis=1)[:,:1500,:]
#split_inputs = np.concatenate([split_inputs_fft,split_inputs], axis=2)
p = np.random.permutation(len(split_inputs))
split_inputs, split_labels = split_inputs[p], split_labels[p]
train_num = int(len(split_inputs)*ratio)
train_X, test_X = split_inputs[train_num:], split_inputs[:train_num]
train_y, test_y = split_labels[train_num:], split_labels[:train_num]
train_X.shape, train_y.shape

((3356, 1500, 1), (3356, 1))

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Reshape, MaxPooling1D, GlobalAveragePooling1D, Dropout
model = Sequential()
model.add(Conv1D(10, 10, activation='relu', input_shape=(sample_size, num_channels)))
#model.add(Conv1D(10, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(20, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Conv1D(40, 10, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
#model.add(Dense(10, activation='linear'))
model.add(Dense(1, activation='linear'))
model.summary(), model.input_shape, model.output_shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_25 (Conv1D)           (None, 1491, 10)          110       
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 497, 10)           0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 488, 20)           2020      
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 162, 20)           0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 162, 20)           0         
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 153, 40)           8040      
_________________________________________________________________
global_average_pooling1d_9 ( (None, 40)                0         
__________

(None, (None, 1500, 1), (None, 1))

In [23]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='logs/{}'.format(time.time()))
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(train_X, train_y, epochs=200, callbacks=[tensorboard], verbose=1, validation_data=(test_X, test_y))


Train on 3356 samples, validate on 838 samples
Epoch 1/200
3356/3356 [==============================] - 2s 641us/step - loss: 3.7193 - val_loss: 2.4750
Epoch 2/200
3356/3356 [==============================] - 0s 146us/step - loss: 2.6597 - val_loss: 2.4784
Epoch 3/200
3356/3356 [==============================] - 1s 280us/step - loss: 2.5190 - val_loss: 2.3095
Epoch 4/200
3356/3356 [==============================] - 1s 180us/step - loss: 2.5286 - val_loss: 2.3563
Epoch 5/200
3356/3356 [==============================] - 1s 153us/step - loss: 2.5038 - val_loss: 2.3080
Epoch 6/200
3356/3356 [==============================] - 1s 165us/step - loss: 2.4833 - val_loss: 2.3393
Epoch 7/200
3356/3356 [==============================] - 1s 186us/step - loss: 2.4749 - val_loss: 2.4869
Epoch 8/200
3356/3356 [==============================] - 1s 150us/step - loss: 2.4173 - val_loss: 2.5632
Epoch 9/200
3356/3356 [==============================] - 1s 150us/step - loss: 2.4053 - val_loss: 2.2683
Epoch 10

In [102]:
max(i[0] for i in test_y)

7.01

In [129]:
split_inputs[0]

array([[2.47841184, 2.44889173],
       [2.51130776, 2.82505028],
       [2.64193073, 2.71464416],
       ...,
       [3.64221877, 3.31960811],
       [2.96314057, 2.79349895],
       [2.74572211, 2.35004836]])

In [130]:
split_labels[]

array([[5.01],
       [5.07],
       [5.1 ],
       ...,
       [4.76],
       [4.07],
       [5.16]])